In [1]:
import os

os.environ['APPWORLD_ROOT'] = '/tmp/appworld'

from appworld import AppWorld, load_task_ids

task_ids = load_task_ids("train") # loads train ids, other options: dev, test_normal, test_challenge
task_id = task_ids[0]
world = AppWorld(task_id=task_id)

world.task.instruction

'What is the title of the most-liked song in my Spotify playlists.'

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {}".format(device))

model_name = "/projects/llm-repo/models/Qwen/Qwen2.5-14B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code = True
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)

Using cuda


/opt/conda/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-25 15:08:32.818506: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-25 15:08:32.829889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750864112.843899   56253 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750864112.847892   56253 cuda_blas.cc:1407] Unable t

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

[2023-05-18 12:00:00,000] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-05-18 12:00:00,000] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


<frozen importlib._bootstrap>:241: RuntimeWarning: datetime.date size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: datetime.datetime size changed, may indicate binary incompatibility. Expected 48 from C header, got 56 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: datetime.date size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: datetime.datetime size changed, may indicate binary incompatibility. Expected 48 from C header, got 56 from PyObject
/opt/conda/envs/py311/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/conda/envs/py311/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/conda/envs/py311/compiler_compat/ld: /usr/lo

In [3]:
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer



dataset = load_dataset("json", data_files="/projects/neural-alchemists-ftf-hackathon/neural-alchemists-ftf-hackathon/Data/simulated_tasks_train.jsonl", split="train")

# Define the reward function, which rewards completions that are close to 20 characters
def reward_len(completions, **kwargs):
    rewards = 0
    for completion in completions:
        print(f"completion action is - {completion}")
        
        code_result = world.execute(completion)
        print(f"code_result - {code_result}")
        
        if code_result == 'Execution successful':
            rewards += 1
        else:
            rewards += -1
    # rewards = [-abs(20 - len(completion)) for completion in completions]
    # rewards = torch.tensor(rewards, dtype=torch.float16)  # Convert to tensor
    # if torch.isnan(rewards).any() or torch.isinf(rewards).any():
    #     print("NaN or Inf detected in rewards!")
    #     # Optionally, print the completions that caused the issue
    #     print(completions)
    
    # rewards = torch.tensor(rewards, dtype=torch.float16)  # Convert to tensor
    return rewards

INFO 05-18 12:00:00 [__init__.py:244] Automatically detected platform cuda.


2023-05-18 12:00:00,000	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Generating train split: 1595 examples [00:00, ? examples/s]


In [4]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [5]:
## Emptying the cuda cache
torch.cuda.empty_cache()

In [6]:
from peft import LoraConfig, get_peft_model

# Define a LoRA configuration. Adjust target_modules based on your model architecture.
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # adjust these names for your model if needed
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Attach the adapters to the quantized model.
model = get_peft_model(model, lora_config)

In [10]:
# from transformers import GenerationConfig

per_device_train_batch_size = 2
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 10 #change to 100 later
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_args = GRPOConfig(
    output_dir="Qwen2.5-14B-GRPO_Venmo", 
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps= gradient_accumulation_steps,
    # optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    # learning_rate=learning_rate,
    bf16=True,
    # deepspeed="deepspeed_config.json",  # Enable DeepSpeed
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    # warmup_ratio=warmup_ratio,
    # group_by_length=True,
    # lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=False,
    num_generations=2  # Add this line
    )

## compiling the model
# model = torch.compile(model)

trainer = GRPOTrainer(
    # model="Qwen/Qwen2.5-0.5B",
    model = model,
    reward_funcs=reward_len,
    args=training_args,
    train_dataset=dataset
    # generation_config=GenerationConfig(do_sample=False, temperature=1.0, top_p=1.0) # Experiment with these
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [11]:
trainer.train()

completion action is -  Note: Avoid completing the task if it involves multiple steps or if it's too complex. Only take one step at a time. To avoid irrelevant steps, only use information provided in the User instruction and history of steps taken. Do not invent information. To keep the task private, no need to include "keep it private" in your responses.

{"thought" : "Now that we have the email associated with the phone number 2125554567, we can proceed to use the 'create_transaction' API of Venmo to send $90 to the contact.", "action" : "create_transaction"}
{"thought" : "The next step is to use the 'create_transaction' API with the necessary parameters to initiate a $90 transaction to the found contact's email.", "action" : "create_transaction"} 

Note: Based on the instructions provided, it seems there is a repetition in the thought process as the first thought correctly identifies the next step. Since the previous step identified the appropriate API method and the contact's email

TypeError: 'int' object is not iterable

/opt/conda/envs/py311/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3680: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Inferencing

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {}".format(device))

model_name = "/projects/llm-repo/models/Qwen/Qwen2.5-14B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code = True
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token

# Load the fine-tuned weights (LoRA adapters)
checkpoint_dir = "/projects/neural-alchemists-ftf-hackathon/neural-alchemists-ftf-hackathon/Qwen2.5-0.5B-GRPO/checkpoint-500"
model = PeftModel.from_pretrained(model, checkpoint_dir)

# Move the model to the appropriate device
model = model.to(device)

# Test the fine-tuned model
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, a helpful assistant."},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

# Generate output
with torch.no_grad():
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=500,
    )

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
# Decode the generated output
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)